## **Probabilistic Atlas Bundle**

### Import libraries

In [ ]:
from fury.colormap import colormap_lookup_table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nibabel as nib
from nibabel.streamlines.array_sequence import concatenate
from dipy.io.image import load_nifti, save_nifti
from dipy.io.streamline import load_trk
from dipy.tracking.streamline import (unlist_streamlines, relist_streamlines,
                                      transform_streamlines, values_from_volume,
                                      set_number_of_points)

from fury import window, actor

### Prepare data

In [ ]:
in_dir = 
out_dir = 
subjects = pd.read_csv(in_dir + 'subjects.csv')
n_subject = subjects.shape[0]
tract = 'AF_L'
n_points = 20
save_atlas = False

# Load MNI template """
image, affine = load_nifti('../data/mni_template/mni_icbm152_t1_tal_nlin_asym_09a.nii')
dims = np.asarray(image.shape)
affine_inv = np.linalg.inv(affine)

### Construct probabilistic atlas

In [ ]:
# Loop through subjects """
dims_mask = np.concatenate((dims, [n_subject]))
mask = np.zeros(dims_mask)

for ind, sub in enumerate(subjects.subject):
    # Load trk file
    file = f'{in_dir}/{sub}/rec_bundles/moved_{tract}__recognized.trk'
    bundle_obj = load_trk(file, reference='same', bbox_valid_check=False)

    # Get streamlines
    points, offset = unlist_streamlines(bundle_obj.streamlines)
    streamlines = relist_streamlines(points, offset)
    streamlines = set_number_of_points(streamlines, n_points)

    # Transform coordinates to voxel space
    streamlines_vox = transform_streamlines(streamlines, affine_inv)

    # Round voxel coordinates and cast them to int
    points, offsets = unlist_streamlines(streamlines_vox)
    streamlines_ind = np.round(points).astype('int')

    # Create coverage mask
    n_point = streamlines_ind.shape[0]
    for i_stream in range(n_point):
        i = streamlines_ind[i_stream, 0]
        j = streamlines_ind[i_stream, 1]
        k = streamlines_ind[i_stream, 2]

        mask[i, j, k, ind] = 1

# Combine subject coverage masks
prob_atlas = mask.sum(axis=3)/n_subject

# Save the atlas
if save_atlas:
    file = f'{out_dir}/prob_atlas_{tract}.nii'
    save_nifti(file, prob_atlas, affine)

In [ ]:
# Plot atlas

### Map probabilistic map to streamlines

In [ ]:
""" Merge all streamlines """
merged_bundles = concatenate(streamlines_all, axis=0)

""" Map data to streamline space """
cov = values_from_volume(prob_atlas, merged_bundles, affine)
mean_cov = np.float_(list(map(np.mean, cov)))

""" Plot mean coverage distribution """
plt.hist(mean_cov)
plt.xlabel('mean coverage')
plt.ylabel('streamlines')

""" Render all streamlines + map density values to each segment """
scene = window.Scene()
scene.SetBackground(1, 1, 1)
aux = transform_streamlines(merged_bundles, affine_inv)
stream_actor = actor.line(aux, prob_atlas, linewidth=0.1, opacity=0.4)
scene.add(stream_actor)
bar = actor.scalar_bar()
scene.add(bar)
window.show(scene)

""" Render all streamlines using color from mean_cov """
scene.clear()
scene.SetBackground(1, 1, 1)
# Adjust range of the colormap (rainbow) to min/max coverage values
lut_cmap = colormap_lookup_table(scale_range=(mean_cov.min(), mean_cov.max()))
# Blue colormap
# hue = (0.5, 0.5)  # blue only
# saturation = (0.0, 1.0)  # black to white

# lut_cmap = actor.colormap_lookup_table(
#     scale_range=(mean_cov.min(), mean_cov.max()),
#     hue_range=hue,
#     saturation_range=saturation)

# Create streamlines
stream_actor = actor.line(aux, mean_cov.astype('float'), linewidth=0.1,
                          lookup_colormap=lut_cmap, opacity=0.2)
scene.add(stream_actor)
# Add colorbar
bar = actor.scalar_bar(lut_cmap)
scene.add(bar)
# Render
window.show(scene)
